In [29]:
!pip install transformers==4.40.0
!pip install -q torch torchvision pydicom opencv-python Pillow accelerate

#GPU update: pip install torch torchvision transformers==4.40.0 pydicom opencv-python Pillow accelerate


In [30]:
# Install dependencies
#Imports
import os, glob
import torch
import pydicom
import cv2
from PIL import Image
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForCausalLM



In [31]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "StanfordAIMI/CheXagent-2-3b"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    low_cpu_mem_usage=True
)
##device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
###########seems to thin that calling the if torch.cudo.is_available() as a param should work!!
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define query
question = "Does this chest X-ray show a pneumothorax?"


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [32]:
for name, param in model.named_parameters():
    if param.device.type == "meta":
        print(f"{name} is on the meta device")

model.layers.28.self_attn.q_proj.weight is on the meta device
model.layers.28.self_attn.q_proj.bias is on the meta device
model.layers.28.self_attn.k_proj.weight is on the meta device
model.layers.28.self_attn.k_proj.bias is on the meta device
model.layers.28.self_attn.v_proj.weight is on the meta device
model.layers.28.self_attn.v_proj.bias is on the meta device
model.layers.28.self_attn.dense.weight is on the meta device
model.layers.28.self_attn.dense.bias is on the meta device
model.layers.28.mlp.fc1.weight is on the meta device
model.layers.28.mlp.fc1.bias is on the meta device
model.layers.28.mlp.fc2.weight is on the meta device
model.layers.28.mlp.fc2.bias is on the meta device
model.layers.28.input_layernorm.weight is on the meta device
model.layers.28.input_layernorm.bias is on the meta device
model.layers.29.self_attn.q_proj.weight is on the meta device
model.layers.29.self_attn.q_proj.bias is on the meta device
model.layers.29.self_attn.k_proj.weight is on the meta device
mo

In [14]:
# attach the colab project to teh files
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [16]:
#Set paths
from pathlib import Path
ROOT = Path('/content/drive/MyDrive/MLProjects/foundation-models-radiology')
DICOM_DIR = ROOT / 'PTXHeadtoHeadSmall'   # use the exact folder name as on Drive
print("exists:", ROOT.exists())

#ROOT = Path("/content/drive/MyDrive")  # adjust if needed
#DICOM_DIR = ROOT / "PTXHeadtoHeadSmall"
JPEG_DIR = Path("/content/drive/MyDrive/MLProjects/foundation-models-radiology/cxr_jpegs")
JPEG_DIR.mkdir(exist_ok=True)

#Replace with: direct file access or use rclone to sync your Google Drive into the pod’s local filesystem.
#RunPod doesn’t support drive.mount().
#GPU update:
#ROOT = Path('/workspace/MLProjects/foundation-models-radiology')
#JPEG_DIR = ROOT / 'cxr_jpegs'



exists: True


In [17]:
# Convert DICOMs to JPEGs
def dicom_to_jpeg(dicom_path, jpeg_path):
    ds = pydicom.dcmread(str(dicom_path))
    img = ds.pixel_array
    img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    img = cv2.equalizeHist(img)  # optional contrast enhancement
    cv2.imwrite(str(jpeg_path), img)

# Batch convert
#dicom_paths = list(DICOM_DIR.rglob("*.dcm"))
# find DICOMs (case-insensitive .dcm)
dicom_paths = list(DICOM_DIR.rglob('*.[dD][cC][mM]'))
print("count:", len(dicom_paths))
print("samples:", dicom_paths[:5])

jpeg_paths = []
for dcm_path in dicom_paths:
    jpg_path = JPEG_DIR / f"{dcm_path.stem}.jpg"
    dicom_to_jpeg(dcm_path, jpg_path)
    jpeg_paths.append(str(jpg_path))


count: 2
samples: [PosixPath('/content/drive/MyDrive/MLProjects/foundation-models-radiology/PTXHeadtoHeadSmall/0a1d53cc79cb2ee50afa507d425f6d15/04797ae93ba9e94174bd928999395887/204611bfbd3e2c990bc2e9d6e131d46a/813f8b5f7031f381d6aadb2edcdd15cf.dcm'), PosixPath('/content/drive/MyDrive/MLProjects/foundation-models-radiology/PTXHeadtoHeadSmall/ffac9214400053aa4c78126255328235/113cd0ccb7393ecb0a5a8a94dac38f72/73cedef0efa5a294616db1be804bd026/3dc870d5ffa18cb81c81342ca478eb4f.dcm')]


In [19]:
# Run inference on each image
def ask_chexagent(image_path, question):
    query = tokenizer.from_list_format([
        {"image": image_path},
        {"text": question}
    ])
    conversation = [
        {"from": "system", "value": "You are a helpful assistant."},
        {"from": "human", "value": query}
    ]

    inputs = tokenizer.apply_chat_template(conversation, add_generation_prompt=True, return_tensors="pt")
    #This returns a dictionary with: input_ids: tokenized input attention_mask: mask for padding
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    output = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        do_sample=False,
        num_beams=1,
        temperature=1.0,
        top_p=1.0,
        use_cache=True,
        max_new_tokens=128
    )[0]

    response = tokenizer.decode(output, skip_special_tokens=True)
    #response = tokenizer.decode(output[input_ids.size(1):-1])
    return response.strip()

In [20]:
# Run on all images and collect responses
import csv

results = []
for path in jpeg_paths:
    answer = ask_chexagent(path, question)
    results.append((path, answer))
    print(f" {Path(path).name} → {answer}")

with open("chexagent_results.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["Image", "Answer"])
    writer.writerows(results)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


NotImplementedError: Cannot copy out of meta tensor; no data!